### Import Libraries

In [2]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import sys
import csv

import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import io
import warnings
warnings.filterwarnings('ignore')


In [3]:
# Load the data files
team_stats = pd.read_csv('team_stats.csv')
match_res = pd.read_csv('match_res.csv')
player_stat = pd.read_csv('player_stat.csv')

In [4]:
#display the head of data files
print(team_stats.columns)
team_stats.head()


Index(['team_name', 'seasons', 'Matches Played', 'wins', 'draws', 'losses',
       'Finishing Position', 'highest score', 'Biggest Winning Margin',
       'Total Raids', 'Successful Raids', 'unsuccessful raid', 'empty raids',
       'success raid %', 'no. of super raids', 'raid touch points',
       'raid bonus points', 'total raid points', 'total tackles',
       'successful tackles', 'unsuccesful tackles', 'successful tackle %',
       'no. of super tackles', 'all outs inflicted', 'total all out points',
       'total defence points'],
      dtype='object')


,team_name,seasons,Matches Played,wins,draws,losses,Finishing Position,highest score,Biggest Winning Margin,Total Raids,...,raid bonus points,total raid points,total tackles,successful tackles,unsuccesful tackles,successful tackle %,no. of super tackles,all outs inflicted,total all out points,total defence points
0,Bengal Warriors,Overall,124,55,15,54,-,48,31,5269,...,483,2208,2812,1017,1795,39%,98,130,260,1110
1,Bengal Warriors,Season 7,19,12,3,4,1,48,31,782,...,76,398,449,174,275,41%,12,31,62,184
2,Bengal Warriors,Season 6,23,12,2,9,2,44,16,963,...,95,436,508,186,322,39%,16,23,46,200
3,Bengal Warriors,Season 5,24,11,6,7,1,44,20,1079,...,122,455,521,181,340,37%,12,27,54,193
4,Bengal Warriors,Season 4,14,3,2,9,8,34,8,602,...,48,202,318,117,201,39%,8,11,22,125


In [5]:
print(match_res.columns)
print(match_res.shape)
match_res.head()

Index(['Season', 'Date', 'Time', 'Status', 'Team1', 'Score1', 'Team2',
       'Score2', 'Venue', 'Match-Results'],
      dtype='object')
(515, 10)


,Season,Date,Time,Status,Team1,Score1,Team2,Score2,Venue,Match-Results
0,Season 6,"Sunday, 07 October",20:00 (IST),Match 1 - Completed,Tamil Thalaivas,42,Patna Pirates,26,"Jawaharlal Nehru Indoor Stadium, Chennai",Tamil Thalaivas beat Patna Pirates (42-26)
1,Season 6,"Sunday, 07 October",21:00 (IST),Match 2 - Completed,Puneri Paltan,32,U Mumba,32,"Jawaharlal Nehru Indoor Stadium, Chennai",Match Tied
2,Season 6,"Monday, 08 October",20:00 (IST),Match 3 - Completed,Puneri Paltan,34,Haryana Steelers,22,"Jawaharlal Nehru Indoor Stadium, Chennai",Puneri Paltan beat Haryana Steelers (34-22)
3,Season 6,"Monday, 08 October",21:00 (IST),Match 4 - Completed,Tamil Thalaivas,32,U.P. Yoddha,37,"Jawaharlal Nehru Indoor Stadium, Chennai",U.P. Yoddha beat Tamil Thalaivas (37-32)
4,Season 6,"Tuesday, 09 October",20:00 (IST),Match 5 - Completed,Dabang Delhi K.C.,32,Gujarat Fortunegiants,32,"Jawaharlal Nehru Indoor Stadium, Chennai",Match Tied


In [6]:
print(player_stat.columns)
player_stat.head()

Index(['Season_Name', 'Metrics', 'Player_Name', 'Team_Name',
       'Total_matches_played', 'Points', 'Player_type'],
      dtype='object')


,Season_Name,Metrics,Player_Name,Team_Name,Total_matches_played,Points,Player_type
0,SEASON 7,Total Points,Pawan Kumar Sehrawat,bengaluru bulls,19,256,Raider
1,SEASON 7,Total Points,Pardeep Narwal,patna pirates,19,243,Raider
2,SEASON 7,Total Points,Naveen Kumar,dabang delhi,19,243,Raider
3,SEASON 7,Total Points,Maninder Singh,bengal warriors,19,192,Raider
4,SEASON 7,Total Points,Siddharth Sirish Desai,telugu titans,18,177,Raider


In [7]:
# filter the matches played by Jaipur Pink Panthers	
df = match_res[(match_res['Team1'] == 'Jaipur Pink Panthers') | (match_res['Team2'] == 'Jaipur Pink Panthers')]
Jaipur_Pink_Panthers = df.iloc[:]
Jaipur_Pink_Panthers.head()

,Season,Date,Time,Status,Team1,Score1,Team2,Score2,Venue,Match-Results
6,Season 6,"Wednesday, 10 October",20:00 (IST),Match 7 - Completed,U Mumba,39,Jaipur Pink Panthers,32,"Jawaharlal Nehru Indoor Stadium, Chennai",U Mumba beat Jaipur Pink Panthers (39-32)
17,Season 6,"Tuesday, 16 October",21:00 (IST),Match 18 - Completed,Haryana Steelers,33,Jaipur Pink Panthers,36,"Motilal Nehru School of Sports, Rai, Sonepat",Jaipur Pink Panthers beat Haryana Steelers (36...
23,Season 6,"Friday, 19 October",21:00 (IST),Match 24 - Completed,Puneri Paltan,29,Jaipur Pink Panthers,25,"Shree Shiv Chhatrapati Sports Complex, Pune",Puneri Paltan beat Jaipur Pink Panthers (29-25)
32,Season 6,"Friday, 26 October",20:00 (IST),Match 33 - Completed,Patna Pirates,41,Jaipur Pink Panthers,30,"Patliputra Sports Complex, Patna",Patna Pirates beat Jaipur Pink Panthers (41-30)
34,Season 6,"Saturday, 27 October",20:00 (IST),Match 35 - Completed,Jaipur Pink Panthers,28,Bengal Warriors,39,"Patliputra Sports Complex, Patna",Bengal Warriors beat Jaipur Pink Panthers (39-28)


In [8]:
Jaipur_Pink_Panthers['Match_Year'] = Jaipur_Pink_Panthers['Season'].replace({"Season 1" : 2014, "Season 2" : 2015, "Season 3" : 2016, "Season 4" : 2016, "Season 5" : 2017, "Season 6" : 2018, "Season 7": 2019})
Jaipur_Pink_Panthers['Match_Year'] = Jaipur_Pink_Panthers['Match_Year'].astype(int)
Jaipur_Pink_Panthers_2014 = Jaipur_Pink_Panthers[Jaipur_Pink_Panthers['Match_Year']>=2014]
Jaipur_Pink_Panthers_2014.count()

Season           104
Date             104
Time             104
Status           104
Team1            104
Score1           104
Team2            104
Score2           104
Venue            104
Match-Results    104
Match_Year       104
dtype: int64

In [9]:
# Combine the teams participating in the pro kabbadi
kabaddi_teams = ['Bengal Warriors', 'Bengaluru Bulls', 'Dabang Delhi K.C.','Gujarat Fortunegiants', 'Haryana Steelers', 'Jaipur Pink Panthers', 'Patna Pirates',
                 'Puneri Paltan', 'Tamil Thalaivas', 'Telugu Titans','U Mumba', 'U.P. Yoddha']
df_team_1 = match_res[match_res['Team1'].isin(kabaddi_teams)]
df_team_2 = match_res[match_res['Team2'].isin(kabaddi_teams)]
df_teams = pd.concat((df_team_1,df_team_2))
df_teams.drop_duplicates()
df_teams.count()

Season           1030
Date             1030
Time             1030
Status           1030
Team1            1030
Score1           1030
Team2            1030
Score2           1030
Venue            1030
Match-Results    1030
dtype: int64

In [10]:
df_teams.head()

,Season,Date,Time,Status,Team1,Score1,Team2,Score2,Venue,Match-Results
0,Season 6,"Sunday, 07 October",20:00 (IST),Match 1 - Completed,Tamil Thalaivas,42,Patna Pirates,26,"Jawaharlal Nehru Indoor Stadium, Chennai",Tamil Thalaivas beat Patna Pirates (42-26)
1,Season 6,"Sunday, 07 October",21:00 (IST),Match 2 - Completed,Puneri Paltan,32,U Mumba,32,"Jawaharlal Nehru Indoor Stadium, Chennai",Match Tied
2,Season 6,"Monday, 08 October",20:00 (IST),Match 3 - Completed,Puneri Paltan,34,Haryana Steelers,22,"Jawaharlal Nehru Indoor Stadium, Chennai",Puneri Paltan beat Haryana Steelers (34-22)
3,Season 6,"Monday, 08 October",21:00 (IST),Match 4 - Completed,Tamil Thalaivas,32,U.P. Yoddha,37,"Jawaharlal Nehru Indoor Stadium, Chennai",U.P. Yoddha beat Tamil Thalaivas (37-32)
4,Season 6,"Tuesday, 09 October",20:00 (IST),Match 5 - Completed,Dabang Delhi K.C.,32,Gujarat Fortunegiants,32,"Jawaharlal Nehru Indoor Stadium, Chennai",Match Tied


In [11]:
df_team_2014 = df_teams.drop(['Date','Time','Venue','Status','Season'],axis = 1)
df_team_2014.head()

,Team1,Score1,Team2,Score2,Match-Results
0,Tamil Thalaivas,42,Patna Pirates,26,Tamil Thalaivas beat Patna Pirates (42-26)
1,Puneri Paltan,32,U Mumba,32,Match Tied
2,Puneri Paltan,34,Haryana Steelers,22,Puneri Paltan beat Haryana Steelers (34-22)
3,Tamil Thalaivas,32,U.P. Yoddha,37,U.P. Yoddha beat Tamil Thalaivas (37-32)
4,Dabang Delhi K.C.,32,Gujarat Fortunegiants,32,Match Tied


In [12]:
# Building the model

df_team_2014 = df_team_2014.reset_index(drop = True)
df_team_2014 = df_team_2014[df_team_2014['Match-Results'] != 'Match Tied']
df_team_2014['Winning_Team'] = df_team_2014['Match-Results'].str.split('beat').str[0]
df_team_2014.head()

,Team1,Score1,Team2,Score2,Match-Results,Winning_Team
0,Tamil Thalaivas,42,Patna Pirates,26,Tamil Thalaivas beat Patna Pirates (42-26),Tamil Thalaivas
2,Puneri Paltan,34,Haryana Steelers,22,Puneri Paltan beat Haryana Steelers (34-22),Puneri Paltan
3,Tamil Thalaivas,32,U.P. Yoddha,37,U.P. Yoddha beat Tamil Thalaivas (37-32),U.P. Yoddha
5,Tamil Thalaivas,28,Telugu Titans,33,Telugu Titans beat Tamil Thalaivas (33-28),Telugu Titans
6,U Mumba,39,Jaipur Pink Panthers,32,U Mumba beat Jaipur Pink Panthers (39-32),U Mumba


In [13]:
# The Prediction label : the winning team column will show "1" if Team 1 has won and "2" if Team 2 has won.
df_team_2014.loc[df_team_2014['Match-Results'] == df_team_2014.Team1,'Winning_Team'] == 1
df_team_2014.loc[df_team_2014['Match-Results'] == df_team_2014.Team2,'Winning_Team'] == 2
df_team_2014 = df_team_2014.drop(['Match-Results'],axis = 1)
df_team_2014.head()

,Team1,Score1,Team2,Score2,Winning_Team
0,Tamil Thalaivas,42,Patna Pirates,26,Tamil Thalaivas
2,Puneri Paltan,34,Haryana Steelers,22,Puneri Paltan
3,Tamil Thalaivas,32,U.P. Yoddha,37,U.P. Yoddha
5,Tamil Thalaivas,28,Telugu Titans,33,Telugu Titans
6,U Mumba,39,Jaipur Pink Panthers,32,U Mumba


In [21]:
# Converting team1 and team2 from categorical variables to continuous inputs

final = pd.get_dummies(df_team_2014, prefix = ['Team1','Team2'],columns = ['Team1','Team2'])

# separate X and y sets
X = final.drop(['Winning_Team'],axis =1)
y = final["Winning_Team"]

# Divide the data for training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 100)


In [22]:
# Display the first 5 rows of the data
final.head()


,Score1,Score2,Winning_Team,Team1_Bengal Warriors,Team1_Bengaluru Bulls,Team1_Dabang Delhi K.C.,Team1_Gujarat Fortunegiants,Team1_Haryana Steelers,Team1_Jaipur Pink Panthers,Team1_Patna Pirates,...,Team2_Dabang Delhi K.C.,Team2_Gujarat Fortunegiants,Team2_Haryana Steelers,Team2_Jaipur Pink Panthers,Team2_Patna Pirates,Team2_Puneri Paltan,Team2_Tamil Thalaivas,Team2_Telugu Titans,Team2_U Mumba,Team2_U.P. Yoddha
0,42,26,Tamil Thalaivas,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,34,22,Puneri Paltan,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,32,37,U.P. Yoddha,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,28,33,Telugu Titans,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,39,32,U Mumba,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [23]:
# Train the model using the logistic regression
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
score = logreg.score(X_train,y_train)
score2 = logreg.score(X_test,y_test)

print("Training set accuracy: ",'%.3f'%(score))
print("Test set accuracy: ",'%.3f'%(score2))

Training set accuracy:  0.733
Test set accuracy:  0.620


In [68]:
# Adding the Standings and the fixures

#Fixure
fixures = pd.read_csv('match_sch.csv')

In [39]:
# Ranking
ranking = pd.read_csv('team_rank.csv')

In [69]:
fixures_ex = fixures[(fixures['Team1']!='Rank 3') & (fixures['Team1']!='Winner of Semi-Final 1') & (fixures['Team1']!='Rank 4')& (fixures['Team1']!='Rank 1')& (fixures['Team1']!='Rank 2')]

In [70]:
fixures_ex.Team1.unique()

array(['Telugu Titans', 'Bengaluru Bulls', 'U Mumba', 'Puneri Paltan',
       'U.P. Yoddha', 'Dabang Delhi K.C.', 'Jaipur Pink Panthers',
       'Tamil Thalaivas', 'Bengal Warriors', 'Haryana Steelers',
       'Gujarat Fortunegiants', 'Patna Pirates'], dtype=object)

In [73]:
fixures_ex.head()
fixures_ex.Team1.unique()

array(['Telugu Titans', 'Bengaluru Bulls', 'U Mumba', 'Puneri Paltan',
       'U.P. Yoddha', 'Dabang Delhi K.C.', 'Jaipur Pink Panthers',
       'Tamil Thalaivas', 'Bengal Warriors', 'Haryana Steelers',
       'Gujarat Fortunegiants', 'Patna Pirates'], dtype=object)

In [74]:
ranking.head()

,Season,Rank,Team_Name,Matches_Played,Matches_Won,Matches_Lost,Matches_Drow,Total_Points
0,Season 7,1,Dabang Delhi K.C.,19,15,2,2,82
1,Season 7,2,Bengal Warriors,19,12,4,3,73
2,Season 7,3,Haryana Steelers,19,12,6,1,65
3,Season 7,4,Bengaluru Bulls,19,10,8,1,58
4,Season 7,5,U.P. Yoddha,18,10,6,2,58


In [75]:
# List for storing the group stage games
pred_set = []

# create new columns with ranking position of each team
fixures_ex.insert(1,'first_position', fixures_ex['Team1'].map(ranking.set_index('Team_Name')['Rank']))
fixures_ex.insert(2,'second_position', fixures_ex['Team2'].map(ranking.set_index('Team_Name')['Rank']))
 

In [76]:
fixures_ex.head()

,Season,first_position,second_position,Date,Time,Status,Team1,Score1,Team2,Score2,Venue,Match-Results
0,Season 7,11,6,"Saturday, 20 July",19:30 (IST),Match 1 - Completed,Telugu Titans,25.0,U Mumba,31.0,"Gachibowli Indoor Stadium, Hyderabad",U Mumba beat Telugu Titans (31-25)
1,Season 7,4,10,"Saturday, 20 July",20:30 (IST),Match 2 - Completed,Bengaluru Bulls,34.0,Patna Pirates,32.0,"Gachibowli Indoor Stadium, Hyderabad",Bengaluru Bulls Beat Patna Pirates (34-32 )
2,Season 7,4,8,"Sunday, 21 July",19:30 (IST),Match 3 - Completed,Bengaluru Bulls,24.0,Gujarat Fortunegiants,42.0,"Gachibowli Indoor Stadium, Hyderabad",Gujarat Fortunegiants Beat Bengaluru Bulls (4...
3,Season 7,11,12,"Sunday, 21 July",20:30 (IST),Match 4 - Completed,Telugu Titans,26.0,Tamil Thalaivas,39.0,"Gachibowli Indoor Stadium, Hyderabad",Tamil Thalaivas Beat Telugu Titans (39-26 )
4,Season 7,6,7,"Monday, 22 July",19:30 (IST),Match 5 - Completed,U Mumba,23.0,Jaipur Pink Panthers,42.0,"Gachibowli Indoor Stadium, Hyderabad",Jaipur Pink Panthers Beat U Mumba (42-23 )


In [77]:
pred_set =[]
for index, row in fixures_ex.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team1': row['Team1'], 'Team2': row['Team2'], 'winning_team':None})
    else:
        pred_set.append({'Team1': row['Team2'], 'Team2': row['Team1'], 'winning_team':None})
    
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()

,Team1,Team2,winning_team
0,U Mumba,Telugu Titans,None
1,Bengaluru Bulls,Patna Pirates,None
2,Bengaluru Bulls,Gujarat Fortunegiants,None
3,Telugu Titans,Tamil Thalaivas,None
4,U Mumba,Jaipur Pink Panthers,None


In [46]:
!pip install pandas_sets

In [47]:
import pandas_sets

In [78]:
#Get dummy variables and add missing columns in the main dataset
pred_set=pd.get_dummies(pred_set, prefix=['Team1','Team2'], columns=['Team1','Team2'])
pred_set.head()

,winning_team,Team1_Bengal Warriors,Team1_Bengaluru Bulls,Team1_Dabang Delhi K.C.,Team1_Gujarat Fortunegiants,Team1_Haryana Steelers,Team1_Jaipur Pink Panthers,Team1_Patna Pirates,Team1_Puneri Paltan,Team1_Telugu Titans,...,Team2_Bengaluru Bulls,Team2_Gujarat Fortunegiants,Team2_Haryana Steelers,Team2_Jaipur Pink Panthers,Team2_Patna Pirates,Team2_Puneri Paltan,Team2_Tamil Thalaivas,Team2_Telugu Titans,Team2_U Mumba,Team2_U.P. Yoddha
0,None,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,None,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,None,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,None,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,None,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [79]:
pred_set.columns

Index(['winning_team', 'Team1_Bengal Warriors', 'Team1_Bengaluru Bulls',
       'Team1_Dabang Delhi K.C.', 'Team1_Gujarat Fortunegiants',
       'Team1_Haryana Steelers', 'Team1_Jaipur Pink Panthers',
       'Team1_Patna Pirates', 'Team1_Puneri Paltan', 'Team1_Telugu Titans',
       'Team1_U Mumba', 'Team1_U.P. Yoddha', 'Team2_Bengal Warriors',
       'Team2_Bengaluru Bulls', 'Team2_Gujarat Fortunegiants',
       'Team2_Haryana Steelers', 'Team2_Jaipur Pink Panthers',
       'Team2_Patna Pirates', 'Team2_Puneri Paltan', 'Team2_Tamil Thalaivas',
       'Team2_Telugu Titans', 'Team2_U Mumba', 'Team2_U.P. Yoddha'],
      dtype='object')

In [81]:
missing_cols = set(final.columns) - set(pred_set.columns)
missing_cols

{'Score1',
 'Score2',
 'Team1_Tamil Thalaivas',
 'Team2_Dabang Delhi K.C.',
 'Winning_Team'}

In [82]:
for c in missing_cols:
    pred_set[c]=0
pred_set=pred_set[final.columns]

pred_set=pred_set.drop(['Winning_Team'],axis=1)
pred_set.head()

,Score1,Score2,Team1_Bengal Warriors,Team1_Bengaluru Bulls,Team1_Dabang Delhi K.C.,Team1_Gujarat Fortunegiants,Team1_Haryana Steelers,Team1_Jaipur Pink Panthers,Team1_Patna Pirates,Team1_Puneri Paltan,...,Team2_Dabang Delhi K.C.,Team2_Gujarat Fortunegiants,Team2_Haryana Steelers,Team2_Jaipur Pink Panthers,Team2_Patna Pirates,Team2_Puneri Paltan,Team2_Tamil Thalaivas,Team2_Telugu Titans,Team2_U Mumba,Team2_U.P. Yoddha
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [83]:
fixures.shape[0]

137

In [86]:
match_num = []
winner_data= []
cnt = []
predictions = logreg.predict(pred_set)
num = 0
for i in range(fixures_ex.shape[0]):
    print(backup_pred_set.iloc[i,1]+ " and " + backup_pred_set.iloc[i,0])
    if predictions[i]==1:
        print("winner: " + backup_pred_set.iloc[i, 1])
        match_num.append(num)
        winner_data.append(backup_pred_set.iloc[i, 1])
        cnt.append(1)
    else:
        print("winner: " + backup_pred_set.iloc[i, 0])
        match_num.append(num)
        winner_data.append(backup_pred_set.iloc[i, 0])
        cnt.append(1)
    print("")
    num = num + 1

Telugu Titans and U Mumba
winner: U Mumba

Patna Pirates and Bengaluru Bulls
winner: Bengaluru Bulls

Gujarat Fortunegiants and Bengaluru Bulls
winner: Bengaluru Bulls

Tamil Thalaivas and Telugu Titans
winner: Telugu Titans

Jaipur Pink Panthers and U Mumba
winner: U Mumba

Puneri Paltan and Haryana Steelers
winner: Haryana Steelers

U.P. Yoddha and Bengal Warriors
winner: Bengal Warriors

Telugu Titans and Dabang Delhi K.C.
winner: Dabang Delhi K.C.

Tamil Thalaivas and Dabang Delhi K.C.
winner: Dabang Delhi K.C.

Gujarat Fortunegiants and U.P. Yoddha
winner: U.P. Yoddha

Telugu Titans and Patna Pirates
winner: Patna Pirates

Puneri Paltan and U Mumba
winner: U Mumba

Jaipur Pink Panthers and Bengal Warriors
winner: Bengal Warriors

Haryana Steelers and Dabang Delhi K.C.
winner: Dabang Delhi K.C.

U Mumba and Bengaluru Bulls
winner: Bengaluru Bulls

Tamil Thalaivas and Patna Pirates
winner: Patna Pirates

Puneri Paltan and Bengal Warriors
winner: Bengal Warriors

Jaipur Pink Panthers

In [96]:
Winner_det = pd.DataFrame({'winner_team': winner_data, 'counter': cnt})

In [97]:
print(Winner_det.head())
Winner_det.groupby(by='winner_team').sum().sort_values(by='counter', ascending=False)

       winner_team  counter
0          U Mumba        1
1  Bengaluru Bulls        1
2  Bengaluru Bulls        1
3    Telugu Titans        1
4          U Mumba        1


,counter
winner_team,
Dabang Delhi K.C.,22
Bengal Warriors,20
Haryana Steelers,18
Bengaluru Bulls,16
U.P. Yoddha,14
U Mumba,12
Jaipur Pink Panthers,10
Gujarat Fortunegiants,8
Puneri Paltan,6


In [98]:
semi = [('Dabang Delhi K.C.','Bengal Warriors'),('Haryana Steelers', 'Bengaluru Bulls')]

In [119]:
def clean_and_predict(matches, ranking, final, logreg):
    positions = []
    for match in matches:
        positions.append(ranking.loc[ranking['Team_Name'] == match[0], 'Rank'].iloc[0])
        positions.append(ranking.loc[ranking['Team_Name'] == match[1], 'Rank'].iloc[0])

    pred_set = []
    i = 0
    j = 0
    while i<len(positions):
        dict1 = {}

        if positions[i] < positions[i+1]:
            dict1.update({'Team1' : matches[j][0], 'Team2' : matches[j][1]})
        else: 
            dict1.update({'Team1' : matches[j][1], 'Team2' : matches[j][0]})
        
        pred_set.append(dict1)
        i+=2
        j+=1

    pred_set =pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    pred_set = pd.get_dummies(pred_set, prefix = ['Team1','Team2'], columns = ['Team1','Team2'])

    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    
    pred_set = pred_set[final.columns]
    print(final.columns)
    pred_set = pred_set.drop(['Winning_Team'], axis = 1)

    predictions = logreg.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i,1] + "and" + backup_pred_set.iloc[i,0])
        if predictions[i] == 1:
            print("Winner : "+ backup_pred_set.iloc[i,1])
        else:
            print("Winner : "+ backup_pred_set.iloc[i,0])
            print("")


In [120]:
clean_and_predict(semi, ranking, final, logreg)

Index(['Score1', 'Score2', 'Winning_Team', 'Team1_Bengal Warriors',
       'Team1_Bengaluru Bulls', 'Team1_Dabang Delhi K.C.',
       'Team1_Gujarat Fortunegiants', 'Team1_Haryana Steelers',
       'Team1_Jaipur Pink Panthers', 'Team1_Patna Pirates',
       'Team1_Puneri Paltan', 'Team1_Tamil Thalaivas', 'Team1_Telugu Titans',
       'Team1_U Mumba', 'Team1_U.P. Yoddha', 'Team2_Bengal Warriors',
       'Team2_Bengaluru Bulls', 'Team2_Dabang Delhi K.C.',
       'Team2_Gujarat Fortunegiants', 'Team2_Haryana Steelers',
       'Team2_Jaipur Pink Panthers', 'Team2_Patna Pirates',
       'Team2_Puneri Paltan', 'Team2_Tamil Thalaivas', 'Team2_Telugu Titans',
       'Team2_U Mumba', 'Team2_U.P. Yoddha'],
      dtype='object')
Bengal WarriorsandDabang Delhi K.C.
Winner : Dabang Delhi K.C.

Bengaluru BullsandHaryana Steelers
Winner : Haryana Steelers



In [121]:
finalmatch = [('Dabang Delhi K.C.','Haryana Steelers')]

In [122]:
clean_and_predict(finalmatch, ranking, final, logreg)

Index(['Score1', 'Score2', 'Winning_Team', 'Team1_Bengal Warriors',
       'Team1_Bengaluru Bulls', 'Team1_Dabang Delhi K.C.',
       'Team1_Gujarat Fortunegiants', 'Team1_Haryana Steelers',
       'Team1_Jaipur Pink Panthers', 'Team1_Patna Pirates',
       'Team1_Puneri Paltan', 'Team1_Tamil Thalaivas', 'Team1_Telugu Titans',
       'Team1_U Mumba', 'Team1_U.P. Yoddha', 'Team2_Bengal Warriors',
       'Team2_Bengaluru Bulls', 'Team2_Dabang Delhi K.C.',
       'Team2_Gujarat Fortunegiants', 'Team2_Haryana Steelers',
       'Team2_Jaipur Pink Panthers', 'Team2_Patna Pirates',
       'Team2_Puneri Paltan', 'Team2_Tamil Thalaivas', 'Team2_Telugu Titans',
       'Team2_U Mumba', 'Team2_U.P. Yoddha'],
      dtype='object')
Haryana SteelersandDabang Delhi K.C.
Winner : Dabang Delhi K.C.



**From above analysis ; its clear that Dabang Delhi K.C. will win this tournament with highest points as well**